# 0.Part: import libraries

In [226]:
import numpy as np
import pandas as pd
from tkinter import *
import tkinter.ttk

# 1.Part of the data load

In [229]:
train = pd.read_csv('mnist_train.csv')
test = pd.read_csv('mnist_test.csv')

prepeart data to work

In [231]:
Y_train = train['label'].values
X_train = train.drop(labels=['label'], axis=1)
X_train = X_train / 255.0
X_train = X_train.values

Y_test = test['label'].values
X_test = test.drop(labels=['label'], axis=1)
X_test = X_test / 255.0
X_test = X_test.values

# a convenient type of response
Y_acum = np.zeros((len(X_train), 10))
for i, l in enumerate(Y_train):
    Y_acum[i][l] = 1
Y_train = Y_acum

Y_acum = np.zeros((len(X_test), 10))
for i, l in enumerate(Y_test):
    Y_acum[i][l] = 1
Y_test = Y_acum

# 2.Part of creating and training a model

I use perceptron model with 3 layers for this task

set random seed

In [237]:
np.random.seed(72)

activation function and her derivitev

In [240]:
relu = lambda x: (x >= 0) * x
relu_deriv = lambda x: x >= 0

In [242]:
sigmoid = lambda x: 1 / (1 + np.exp(-x))
sigmoid_deriv = lambda x: sigmoid(x)*(1 - sigmoid(x))

# inicialization weights and other variables

In [245]:
hidden_size = 72
epochs = 5
alpha = 0.005 # step size of gradient descent
batch_size = 72
num_pixels = 784
num_lables = 10
weights_0_1 = np.random.random((num_pixels, hidden_size)) * 0.2 - 0.1
weights_1_2 = np.random.random((hidden_size, num_lables)) * 0.2 - 0.1

prediction generation function

In [248]:
def get_pred(inp):
    layer_hidden = relu(np.dot(inp, weights_0_1))
    pred = np.dot(layer_hidden, weights_1_2)
    return pred

## lerning and testing part

I used dropout regularization to prevent the model from overfitting

In [252]:
for epoch in range(epochs):
    error, correct_cnt = 0.0, 0
    for i in range(len(X_train) // batch_size):
        batch_start, batch_end = batch_size * i, batch_size * (i + 1)
        layer_0 = X_train[batch_start:batch_end]
        layer_1 = relu(np.dot(layer_0, weights_0_1))
        # <dropout>
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        # <\dropout>

        layer_2 = np.dot(layer_1, weights_1_2)

        error += np.sum((Y_train[batch_start:batch_end] - layer_2) ** 2)

        layer_2_delta = (Y_train[batch_start:batch_end] - layer_2)
        layer_1_delta = np.dot(layer_2_delta, weights_1_2.T) * relu_deriv(np.dot(layer_0, weights_0_1))

        # <dropout>
        layer_1_delta *= dropout_mask
        # <\dropout>

        weights_1_2 += np.dot(layer_1.T, layer_2_delta) * alpha
        weights_0_1 += np.dot(layer_0.T, layer_1_delta) * alpha

        if i % (10000 // batch_size) == 0:
            print(f'{round(i * 100 / len(X_train) * batch_size)}%')

    test_error, test_correct_cnt = 0.0, 0
    for i in range(len(X_test)):
        res = get_pred(X_test[i:i + 1])
        test_error += np.sum((Y_test[i:i + 1] - res) ** 2)
        test_correct_cnt += int(np.argmax(res) == np.argmax(Y_test[i:i + 1]))

    for i in range(len(X_train)):
            res = get_pred(X_train[i:i + 1])
            correct_cnt += int(np.argmax(res) == np.argmax(Y_train[i:i + 1]))

    print(f"\nTrain error: {round(error / len(X_train), 3)} "
          f"Train correct %: {round(correct_cnt / len(X_train) * 100, 3)} "
          f"Test error: {round(test_error / len(X_test), 3)} "
          f"Test correct %: {round(test_correct_cnt / len(X_test) * 100, 3)} "
          f"epoch: {epoch + 1} ")

0%
17%
33%
50%
66%
83%
99%

Train error: 0.432 Train correct %: 89.263 Test error: 0.256 Test correct %: 89.85 epoch: 1 
0%
17%
33%
50%
66%
83%
99%

Train error: 0.343 Train correct %: 91.3 Test error: 0.22 Test correct %: 91.9 epoch: 2 
0%
17%
33%
50%
66%
83%
99%

Train error: 0.324 Train correct %: 92.083 Test error: 0.22 Test correct %: 92.3 epoch: 3 
0%
17%
33%
50%
66%
83%
99%

Train error: 0.318 Train correct %: 92.425 Test error: 0.221 Test correct %: 92.69 epoch: 4 
0%
17%
33%
50%
66%
83%
99%

Train error: 0.312 Train correct %: 92.497 Test error: 0.207 Test correct %: 92.66 epoch: 5 


Try to use sigmoid instead relu. And without regularization

In [257]:
def get_pred(inp):
    layer_hidden = sigmoid(np.dot(inp, weights_0_1))
    pred = np.dot(layer_hidden, weights_1_2)
    return pred

for epoch in range(epochs):
    error, correct_cnt = 0.0, 0
    for i in range(len(X_train) // batch_size):
        batch_start, batch_end = batch_size * i, batch_size * (i + 1)
        layer_0 = X_train[batch_start:batch_end]
        layer_1 = sigmoid(np.dot(layer_0, weights_0_1))
        # <dropout>
        # dropout_mask = np.random.randint(2, size=layer_1.shape)
        # layer_1 *= dropout_mask * 2
        # <\dropout>

        layer_2 = np.dot(layer_1, weights_1_2)

        error += np.sum((Y_train[batch_start:batch_end] - layer_2) ** 2)

        layer_2_delta = (Y_train[batch_start:batch_end] - layer_2)
        layer_1_delta = np.dot(layer_2_delta, weights_1_2.T) * sigmoid_deriv(np.dot(layer_0, weights_0_1))

        # <dropout>
        # layer_1_delta *= dropout_mask
        # <\dropout>

        weights_1_2 += np.dot(layer_1.T, layer_2_delta) * alpha
        weights_0_1 += np.dot(layer_0.T, layer_1_delta) * alpha

        if i % (10000 // batch_size) == 0:
            print(f'{round(i * 100 / len(X_train) * batch_size)}%')

    test_error, test_correct_cnt = 0.0, 0
    for i in range(len(X_test)):
        res = get_pred(X_test[i:i + 1])
        test_error += np.sum((Y_test[i:i + 1] - res) ** 2)
        test_correct_cnt += int(np.argmax(res) == np.argmax(Y_test[i:i + 1]))

    for i in range(len(X_train)):
            res = get_pred(X_train[i:i + 1])
            correct_cnt += int(np.argmax(res) == np.argmax(Y_train[i:i + 1]))

    print(f"\nTrain error: {round(error / len(X_train), 3)} "
          f"Train correct %: {round(correct_cnt / len(X_train) * 100, 3)} "
          f"Test error: {round(test_error / len(X_test), 3)} "
          f"Test correct %: {round(test_correct_cnt / len(X_test) * 100, 3)} "
          f"epoch: {epoch + 1} ")

0%
17%
33%
50%
66%
83%
99%

Train error: 0.197 Train correct %: 91.738 Test error: 0.189 Test correct %: 92.22 epoch: 1 
0%
17%
33%
50%
66%
83%
99%

Train error: 0.176 Train correct %: 92.585 Test error: 0.175 Test correct %: 92.89 epoch: 2 
0%
17%
33%
50%
66%
83%
99%

Train error: 0.162 Train correct %: 93.195 Test error: 0.163 Test correct %: 93.29 epoch: 3 
0%
17%
33%
50%
66%
83%
99%

Train error: 0.152 Train correct %: 93.657 Test error: 0.155 Test correct %: 93.68 epoch: 4 
0%
17%
33%
50%
66%
83%
99%

Train error: 0.144 Train correct %: 94.017 Test error: 0.148 Test correct %: 93.9 epoch: 5 


# 3. Paint part

In [260]:
def set_pixel(x, y, size=20, color=100):
    global lable_pred_number, canvas_matrix

    if canvas_matrix[0, x // 20 + y // 20 * 28] < color / 100:
        canvas.create_rectangle(x, y, x + size, y + size, fill=f"gray{color}", width=0)
        canvas_matrix[0, x // 20 + y // 20 * 28] = color / 100

    res = get_pred(canvas_matrix)

    lable_pred_number["text"] = f"Prediction: {res.argmax()}"
    for i in range(10):
        bars_digits[i]["value"] = res[0, i] * 100

In [262]:
ind_num = 0

def paint_number():
    clear_canvas()
    global ind_num, lable_current_number
    num = X_test[ind_num]
    lable_current_number['text'] = f"Current number: {Y_test[ind_num].argmax()}"
    ind_num += 1
    for i in range(784):
        if num[i] != 0:
            x, y = i * 20 % 560, i * 20 // 560 * 20
            set_pixel(x, y, color=round(num[i] * 100))

In [264]:
def draw(event):
    for dx in (-10, 0, 10):
        for dy in (-10, 0, 10):
            x, y = (event.x + dx) // 20 * 20, (event.y + dy) // 20 * 20

            if (x < 0 or y < 0 or x > 540 or y > 540):
                continue

            if dx == 0 and dy == 0:
                set_pixel(x, y, 20, 100)
            else:
                set_pixel(x, y, 20, 50)

In [266]:
def clear_canvas():
    global canvas_matrix
    canvas.delete('all')
    canvas['bg'] = 'black'
    canvas_matrix = np.zeros((1, 784))
    for i in range(10):
        bars_digits[i]["value"] = 0

Drawing window settings and starting drawing

In [269]:
x, y = 0, 0
brush_size = 10
color = "white"
canvas_matrix = np.zeros((1, 784))

root = Tk()
root.title("Digit recognaizer")
root.geometry("720x720")
root.resizable(0, 0)

root.columnconfigure(12, weight=1)
root.rowconfigure(2, weight=1)

canvas = Canvas(root, bg="black", width=555, height=575)
canvas.grid(row=3, column=0, columnspan=6, rowspan=10, padx=5, pady=5)

canvas.bind("<B1-Motion>", draw)

Button(root, text="Clear", width=12, command=clear_canvas).grid(row=2, column=2)
Button(root, text="next num", width=12, command=paint_number).grid(row=2, column=3)
lable_current_number = Label(root, text=f"Current number: {Y_train[ind_num].argmax()}", font="20")
lable_current_number.grid(row=2, column=4)
lable_pred_number = Label(root, text=f"Current number: {-1}", font="20")
lable_pred_number.grid(row=2, column=5)
# digits lables
bars_digits = []
for i in range(10):
    bars_digits.append(tkinter.ttk.Progressbar(orient="horizontal", length=100, value=0, mode="determinate"))
    Label(root, text=f"{i}:", font="20").grid(row=i + 3, column=6)
    bars_digits[i].grid(row=i + 3, column=7)

root.mainloop()